In [50]:
# Importamos librerías necesarias
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

# Lista de URLs de las 49 páginas
webs = ["https://www.padelnuestro.com/palas-padel/?p={}".format(i) for i in range(1, 50)]

# Listas vacías para incluir la distinta información
palas = []
precio = []
enlaces = []
categorias = []
marcas = []
generos = []
formas = []

# Definir listas de palabras clave para cada tipo de categoría
niveles_juego_keywords = ["Principiante", "Intermedio", "Avanzado", "Competición"]
marcas_keywords = [
    "Adidas", "Bullpadel", "Head", "Wilson", "Dunlop", "Drop Shot", "Softee", "Enebe", "Babolat", "Black Crown", "Cartri",
    "Eme", "Gama", "Joma", "Just Ten", "Kaitt", "Kelme", "Kuikma", "Nox", "Orygen", "Royal Padel", "Siux", "Star Vie",
    "Varlion", "Vibor-A", "Varlion", "Wingpadel", "Vision", "Rossignol", "DHS", "Tecnifibre", "Power Padel", "Oxdog",
    "Prokennex", "Prince", "Volkl", "Slazenger", "ShockOut", "RoyalPadel", "Qustom Padel", "Paddle Coach"
]
generos_keywords = ["Hombre", "Mujer", "Unisex"]
formas_keywords = ["Redonda", "Diamante", "Lágrima", "Híbrida", "Polivalente", "Control"]

# Función para extraer detalles de cada producto
def extraer_detalles_producto(url):
    page = requests.get(url)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extraer el nombre del producto
        a = soup.find('span', class_='name')
        if a:
            palas.append(a.text.strip())
        else:
            palas.append("No name")



        # Extraer el precio del producto
        c = soup.select_one('span.price')
        if c:
            precio.append(c.text.strip())
        else:
            precio.append("No price")

        # Extraer las categorías del producto
        categories = soup.find_all('span', class_='category')
        nivel_juego, marca, genero, forma = "No category", "No brand", "No gender", "No shape"

        for category in categories:
            text = category.text.strip()
            if any(keyword in text for keyword in niveles_juego_keywords):
                nivel_juego = text
            elif any(keyword in text for keyword in marcas_keywords):
                marca = text
            elif any(keyword in text for keyword in generos_keywords):
                genero = text
            elif any(keyword in text for keyword in formas_keywords):
                forma = text

        categorias.append(nivel_juego)
        marcas.append(marca)
        generos.append(genero)
        formas.append(forma)

        # Guardar la URL del producto
        enlaces.append(url)
    else:
        print(f"Error al acceder a {url}: Status code {page.status_code}")

# Función para procesar cada página de productos
def procesar_pagina(web):
    page = requests.get(web)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
        product_containers = soup.find_all('div', class_="product-item-info")
        product_links = []
        for container in product_containers:
            enlace_tag = container.find('a', class_='product-item-link')
            if enlace_tag:
                enlace = enlace_tag.get('href')
                if enlace:
                    product_links.append(enlace)
        return product_links
    else:
        print(f"Error al acceder a {web}: Status code {page.status_code}")
        return []

# Usamos concurrent.futures para procesar las páginas de productos concurrentemente
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Primero, obtenemos todos los enlaces de productos
    futures = [executor.submit(procesar_pagina, web) for web in webs]
    product_links = []
    for future in concurrent.futures.as_completed(futures):
        product_links.extend(future.result())

# Usamos concurrent.futures para extraer los detalles de los productos concurrentemente
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(extraer_detalles_producto, link) for link in product_links]
    for future in concurrent.futures.as_completed(futures):
        future.result()

# Crear un DataFrame con los datos obtenidos
df = pd.DataFrame({
    'Pala': palas,
    'Precio': precio,
    'Nivel de Juego': categorias,
    'Marca': marcas,
    'Genero': generos,
    'Forma': formas,
    'Enlace': enlaces
})

# Guardar el DataFrame en un archivo CSV
df.to_csv('palas_padel.csv', index=False)

print("Scraping completado y datos guardados en 'palas_padel.csv'")


Scraping completado y datos guardados en 'palas_padel.csv'


In [52]:
print (df)

                                                  Pala    Precio  \
0                                     BULLPADEL NEURON  172,95 €   
1                                HEAD EXTREME ONE 2023  251,95 €   
2                                 SIUX BEAT HYBRID AIR   80,00 €   
3                      STAR VIE S2 FLUOR CARBON EFFECT  114,95 €   
4                            NOX AT PRO CUP COORP 2024  118,95 €   
..                                                 ...       ...   
928                 RS PADEL PRO EDITION SIMON VASQUEZ  298,95 €   
929                 PICKLEBALL HEAD EXTREME ELITE 2023   69,95 €   
930                 PICKLEBALL HEAD RADICAL PRO 226022  109,95 €   
931  PICKLEBALL ADIDAS PB ADIPOWER ATTK 3.2 GRIS AM...  229,95 €   
932  PICKLEBALL ADIDAS PB ADIPOWER ATTK TEAM NEGRO ...  184,95 €   

                 Nivel de Juego       Marca   Genero      Forma  \
0       /Avanzado / Competición  /Bullpadel  /Hombre   /Híbrida   
1       /Avanzado / Competición       /Head  /Hom

In [45]:
# Nombre del archivo CSV
csv_file = 'palas_padel.csv'

# Nombre del archivo JSON de salida
json_file = 'palas_padel.json'

# Leer el archivo CSV
df = pd.read_csv(csv_file)

# Convertir el DataFrame a JSON
df.to_json(json_file, orient='records', lines=True)

print(f"Archivo CSV '{csv_file}' convertido a JSON y guardado como '{json_file}'")

Archivo CSV 'palas_padel.csv' convertido a JSON y guardado como 'palas_padel.json'
